In [ ]:
# Cell 1: Introduction and Setup
"""
# 🤖 Multi-Model LLM Comparison Tool
## Google Colab Tutorial

Learn how to compare responses from multiple AI models and generate professional PDF reports!

What you'll learn:
- Connect to OpenRouter API (access to many AI models)
- Query multiple models simultaneously
- Compare responses side by side
- Generate professional PDF reports
"""

print("🎓 Welcome to AI Model Comparison with PDF Reports!")
print("Let's compare AI models and create professional documentation!")


In [ ]:
# Cell 2: Install and Import Everything
# Install required packages
!pip install requests reportlab

# Import all required libraries
import requests
import json
import time
import os
import html
import textwrap
from datetime import datetime
from typing import List, Dict

# ReportLab imports for PDF generation
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
from reportlab.lib import colors
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_JUSTIFY

print("✅ All packages installed and imported!")


In [ ]:
# Cell 3: Complete AI Comparison Class
class AIModelComparison:
    def __init__(self, api_key):
        """Initialize with your OpenRouter API key"""
        self.api_key = api_key
        self.base_url = "https://openrouter.ai/api/v1/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        
        # Popular models that work well
        self.models = [
            "anthropic/claude-3.5-sonnet",
            "meta-llama/llama-3.1-70b-instruct",
            "openai/gpt-4.1-mini",
            "anthropic/claude-sonnet-4",
            "google/gemini-2.5-flash",
            "qwen/qwen2.5-vl-72b-instruct"
        ]
        
    def query_model(self, model: str, prompt: str, max_tokens: int = 1500) -> Dict:
        """Query a specific model"""
        payload = {
            "model": model,
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": max_tokens,
            "temperature": 0.7
        }
        
        try:
            response = requests.post(self.base_url, headers=self.headers, json=payload, timeout=30)
            
            if response.status_code == 200:
                data = response.json()
                return {
                    "model": model,
                    "success": True,
                    "response": data["choices"][0]["message"]["content"].strip(),
                    "tokens_used": data.get("usage", {}).get("total_tokens", "N/A"),
                    "error": None
                }
            else:
                return {
                    "model": model,
                    "success": False,
                    "response": f"Error: {response.status_code}",
                    "tokens_used": "N/A",
                    "error": response.text
                }
                
        except Exception as e:
            return {
                "model": model,
                "success": False,
                "response": f"Exception: {str(e)}",
                "tokens_used": "N/A",
                "error": str(e)
            }
    
    def query_multiple_models(self, prompt: str, max_tokens: int = 1500, delay: float = 0.5) -> List[Dict]:
        """Query multiple models with the same prompt"""
        responses = []
        
        print(f"🤖 Querying {len(self.models)} models...")
        print(f"📝 Prompt: {prompt[:100]}...")
        print("=" * 80)
        
        for i, model in enumerate(self.models, 1):
            print(f"[{i}/{len(self.models)}] Querying {model}...", end=" ")
            
            response = self.query_model(model, prompt, max_tokens)
            responses.append(response)
            
            if response["success"]:
                print("✅")
            else:
                print("❌")
            
            # Add delay to avoid rate limiting
            if i < len(self.models):
                time.sleep(delay)
        
        return responses
    
    def display_responses(self, responses: List[Dict], prompt: str):
        """Display responses in a nice readable format"""
        print("\n" + "=" * 120)
        print(f"🕒 RESULTS - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 120)
        print(f"📝 ORIGINAL PROMPT: {prompt}")
        print("=" * 120)
        
        # Calculate column width
        col_width = max(35, max(len(r["model"]) for r in responses) + 5)
        
        # Header
        header = f"{'MODEL':<{col_width}} | {'STATUS':<8} | {'TOKENS':<8} | RESPONSE"
        print(header)
        print("-" * len(header))
        
        for response in responses:
            model_name = response["model"]
            status = "✅ OK" if response["success"] else "❌ FAIL"
            tokens = str(response["tokens_used"])
            resp_text = response["response"]
            
            # Truncate long responses for table view
            if len(resp_text) > 80:
                resp_preview = resp_text[:77] + "..."
            else:
                resp_preview = resp_text
            
            # Replace newlines with spaces for table format
            resp_preview = resp_preview.replace('\n', ' ').replace('\r', ' ')
            
            print(f"{model_name:<{col_width}} | {status:<8} | {tokens:<8} | {resp_preview}")
        
        print("=" * 120)
        
        # Detailed responses
        print("\n📋 DETAILED RESPONSES:")
        print("=" * 120)
        
        for i, response in enumerate(responses, 1):
            print(f"\n[{i}] 🤖 MODEL: {response['model']}")
            print(f"📊 STATUS: {'✅ Success' if response['success'] else '❌ Failed'}")
            print(f"🔢 TOKENS: {response['tokens_used']}")
            print("📄 RESPONSE:")
            print("-" * 60)
            print(response["response"])
            print("-" * 60)

print("✅ AIModelComparison class ready!")


In [ ]:
# Cell 4: PDF Generation Methods

def save_to_pdf_columns(self, responses: List[Dict], prompt: str, filename: str = None):
    """Save responses to PDF with 2-column layout - ENHANCED WITH FALLBACKS"""
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"openrouter_responses_{timestamp}.pdf"
    
    def clean_text_for_pdf(text: str, max_length: int = None) -> str:
        """Advanced text cleaning for PDF generation with length control"""
        if not text:
            return "No content available"
        
        # Convert to string
        text = str(text)
        
        # Remove null bytes and problematic control characters
        text = text.replace('\x00', '').replace('\x08', '').replace('\x0b', '').replace('\x0c', '')
        text = text.replace('\x0e', '').replace('\x0f', '').replace('\x10', '').replace('\x11', '')
        
        # Replace problematic characters with safe alternatives
        replacements = {
            '&': 'and',
            '<': '[',
            '>': ']',
            '"': "'",
            '\\': '/',
            '\u2018': "'",  # Left single quote
            '\u2019': "'",  # Right single quote
            '\u201c': '"',  # Left double quote
            '\u201d': '"',  # Right double quote
            '\u2013': '-',  # En dash
            '\u2014': '-',  # Em dash
            '\u2026': '...',  # Ellipsis
        }
        
        for old, new in replacements.items():
            text = text.replace(old, new)
        
        # Preserve line breaks for readability
        text = text.replace('\n', ' | ')
        
        # Remove non-printable characters but keep basic punctuation
        text = ''.join(char for char in text if ord(char) >= 32 and ord(char) <= 126)
        
        # Apply length limit if specified
        if max_length and len(text) > max_length:
            text = text[:max_length] + "... [truncated]"
        
        return text.strip() or "No content"
    
    def create_enhanced_table_pdf():
        """Create enhanced table-based PDF with smart content handling"""
        try:
            print(f"🎯 Creating enhanced PDF: {filename}")
            
            # Create PDF document with optimized settings
            doc = SimpleDocTemplate(
                filename,
                pagesize=A4,
                rightMargin=25,
                leftMargin=25,
                topMargin=40,
                bottomMargin=40
            )
            
            story = []
            styles = getSampleStyleSheet()
            
            # Custom styles with better sizing
            title_style = ParagraphStyle(
                'EnhancedTitle',
                parent=styles['Title'],
                fontSize=16,
                spaceAfter=15,
                alignment=TA_CENTER,
                textColor=colors.darkblue
            )
            
            heading_style = ParagraphStyle(
                'EnhancedHeading',
                parent=styles['Heading2'],
                fontSize=12,
                spaceAfter=8,
                spaceBefore=12,
                textColor=colors.darkblue
            )
            
            model_style = ParagraphStyle(
                'EnhancedModel',
                parent=styles['Normal'],
                fontSize=9,
                textColor=colors.darkblue,
                alignment=TA_LEFT,
                fontName='Helvetica-Bold',
                wordWrap='LTR'
            )
            
            response_style = ParagraphStyle(
                'EnhancedResponse',
                parent=styles['Normal'],
                fontSize=8,
                alignment=TA_LEFT,
                leftIndent=2,
                rightIndent=2,
                spaceAfter=1,
                wordWrap='LTR'
            )
            
            # Title and metadata
            story.append(Paragraph("Multi-Model AI Comparison Report", title_style))
            story.append(Spacer(1, 10))
            
            # Summary info
            story.append(Paragraph(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", styles['Normal']))
            story.append(Paragraph(f"Total Models: {len(responses)}", styles['Normal']))
            
            successful = sum(1 for r in responses if r["success"])
            story.append(Paragraph(f"Successful: {successful}/{len(responses)}", styles['Normal']))
            story.append(Spacer(1, 12))
            
            # Original prompt (truncated for space)
            story.append(Paragraph("Prompt", heading_style))
            clean_prompt = clean_text_for_pdf(prompt, 500)  # Limit prompt length
            story.append(Paragraph(clean_prompt, styles['Normal']))
            story.append(Spacer(1, 15))
            
            # Responses section
            story.append(Paragraph("Model Responses", heading_style))
            
            # Smart pagination - split responses into chunks if too many
            chunk_size = 10  # Max responses per table to prevent overflow
            response_chunks = [responses[i:i + chunk_size] for i in range(0, len(responses), chunk_size)]
            
            for chunk_idx, chunk in enumerate(response_chunks):
                if chunk_idx > 0:
                    story.append(PageBreak())  # New page for additional chunks
                
                # Prepare table data for this chunk
                table_data = []
                
                # Header row
                table_data.append([
                    Paragraph("Model", model_style),
                    Paragraph("Response", model_style)
                ])
                
                # Data rows with smart length management
                for response in chunk:
                    # Model name with status
                    model_name = response["model"].split('/')[-1]
                    status_symbol = "[OK]" if response["success"] else "[ERR]"
                    tokens = f"({response['tokens_used']})" if response["success"] and response["tokens_used"] != "N/A" else ""
                    
                    model_cell_text = f"{model_name} {status_symbol} {tokens}"
                    model_cell_clean = clean_text_for_pdf(model_cell_text, 100)
                    
                    # Response content with adaptive length based on number of responses
                    max_response_length = 800 if len(chunk) <= 5 else 400  # Longer responses for fewer models
                    
                    if response["success"]:
                        response_text = clean_text_for_pdf(response["response"])
                    else:
                        error_text = response.get("error", response["response"])
                        response_text = f"Error: {clean_text_for_pdf(error_text)}"
                    
                    # Add to table
                    table_data.append([
                        Paragraph(model_cell_clean, model_style),
                        Paragraph(response_text, response_style)
                    ])
                
                # Create table with dynamic column widths
                available_width = A4[0] - 50  # Account for margins
                col_widths = [available_width * 0.25, available_width * 0.75]  # 25/75 split
                
                comparison_table = Table(table_data, colWidths=col_widths, repeatRows=1)
                
                # Enhanced table styling
                table_style = [
                    # Header styling
                    ('BACKGROUND', (0, 0), (-1, 0), colors.darkblue),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
                    ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
                    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                    ('FONTSIZE', (0, 0), (-1, 0), 10),
                    ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
                    ('TOPPADDING', (0, 0), (-1, 0), 8),
                    
                    # Data cells
                    ('BACKGROUND', (0, 1), (-1, -1), colors.white),
                    ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
                    ('ALIGN', (0, 1), (-1, -1), 'LEFT'),
                    ('VALIGN', (0, 1), (-1, -1), 'TOP'),
                    ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
                    ('FONTSIZE', (0, 1), (-1, -1), 8),
                    ('LEFTPADDING', (0, 1), (-1, -1), 5),
                    ('RIGHTPADDING', (0, 1), (-1, -1), 5),
                    ('TOPPADDING', (0, 1), (-1, -1), 6),
                    ('BOTTOMPADDING', (0, 1), (-1, -1), 6),
                    
                    # Borders
                    ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
                    ('LINEBELOW', (0, 0), (-1, 0), 2, colors.darkblue),
                ]
                
                # Add alternating row colors
                for i in range(1, len(table_data)):
                    if i % 2 == 0:
                        table_style.append(('BACKGROUND', (0, i), (-1, i), colors.lightgrey))
                
                comparison_table.setStyle(TableStyle(table_style))
                story.append(comparison_table)
                story.append(Spacer(1, 15))
            
            # Footer
            footer_text = f"Report generated on {datetime.now().strftime('%Y-%m-%d at %H:%M:%S')}"
            story.append(Paragraph(footer_text, styles['Normal']))
            
            # Build PDF
            doc.build(story)
            return filename
            
        except Exception as e:
            print(f"⚠️ Enhanced PDF creation failed: {e}")
            return None
    
    def create_simple_fallback_pdf():
        """Fallback: Create simple text-based PDF"""
        try:
            simple_filename = filename.replace('.pdf', '_simple.pdf')
            print(f"🔄 Creating simple fallback PDF: {simple_filename}")
            
            doc = SimpleDocTemplate(
                simple_filename,
                pagesize=A4,
                rightMargin=40,
                leftMargin=40,
                topMargin=50,
                bottomMargin=50
            )
            
            story = []
            styles = getSampleStyleSheet()
            
            # Title
            story.append(Paragraph("Multi-Model AI Comparison Report", styles['Title']))
            story.append(Spacer(1, 12))
            
            # Basic info
            story.append(Paragraph(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", styles['Normal']))
            story.append(Spacer(1, 10))
            
            # Prompt
            story.append(Paragraph("Prompt:", styles['Heading2']))
            story.append(Paragraph(clean_text_for_pdf(prompt, 1000), styles['Normal']))
            story.append(Spacer(1, 15))
            
            # Responses
            story.append(Paragraph("Responses:", styles['Heading2']))
            
            for i, response in enumerate(responses):
                model_name = response["model"].split('/')[-1]
                status = "Success" if response["success"] else "Failed"
                
                # Model header
                story.append(Paragraph(f"{i+1}. {model_name} - {status}", styles['Heading3']))
                
                # Response content
                if response["success"]:
                    content = clean_text_for_pdf(response["response"], 2000)
                else:
                    content = f"Error: {clean_text_for_pdf(response['response'], 500)}"
                
                story.append(Paragraph(content, styles['Normal']))
                story.append(Spacer(1, 10))
            
            doc.build(story)
            return simple_filename
            
        except Exception as e:
            print(f"⚠️ Simple PDF creation failed: {e}")
            return None
    
    def create_text_fallback():
        """Final fallback: Create text file"""
        try:
            text_filename = filename.replace('.pdf', '.txt')
            print(f"🔄 Creating text fallback: {text_filename}")
            
            with open(text_filename, 'w', encoding='utf-8') as f:
                f.write("Multi-Model AI Comparison Report\n")
                f.write("=" * 50 + "\n\n")
                f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Total Models: {len(responses)}\n\n")
                
                f.write("PROMPT:\n")
                f.write("-" * 20 + "\n")
                f.write(prompt + "\n\n")
                
                f.write("RESPONSES:\n")
                f.write("-" * 20 + "\n\n")
                
                for i, response in enumerate(responses):
                    model_name = response["model"].split('/')[-1]
                    status = "SUCCESS" if response["success"] else "FAILED"
                    
                    f.write(f"{i+1}. {model_name} - {status}\n")
                    f.write("-" * 30 + "\n")
                    
                    if response["success"]:
                        f.write(response["response"])
                    else:
                        f.write(f"Error: {response['response']}")
                    
                    f.write("\n\n" + "="*50 + "\n\n")
            
            return text_filename
            
        except Exception as e:
            print(f"❌ Text fallback failed: {e}")
            return None
    
    # Main execution with cascading fallbacks
    try:
        # Strategy 1: Enhanced table PDF
        result = create_enhanced_table_pdf()
        if result:
            print(f"✅ Enhanced PDF created successfully: {result}")
            print("📋 Features:")
            print("  • Smart content pagination")
            print("  • Adaptive text truncation")
            print("  • Professional 2-column layout")
            print("  • Status indicators and token counts")
            return result
        
        # Strategy 2: Simple PDF fallback
        print("🔄 Trying simple PDF format...")
        result = create_simple_fallback_pdf()
        if result:
            print(f"✅ Simple PDF created: {result}")
            return result
        
        # Strategy 3: Text file fallback
        print("🔄 Trying text file format...")
        result = create_text_fallback()
        if result:
            print(f"✅ Text file created: {result}")
            return result
        
        print("❌ All PDF creation methods failed")
        return None
        
    except Exception as e:
        print(f"❌ Unexpected error in PDF generation: {e}")
        import traceback
        traceback.print_exc()
        return None

# Replace the PDF method with the enhanced version
AIModelComparison.save_to_pdf = save_to_pdf_columns

print("✅ ENHANCED PDF Generator with Fallbacks Ready!")
print("🎯 New Features:")
print("  📊 Smart content pagination (prevents overflow)")
print("  📏 Adaptive text truncation based on content volume")
print("  🔄 Triple fallback system (Enhanced → Simple → Text)")
print("  🛡️ Robust error handling and recovery")
print("  📋 Professional table layouts with alternating colors")
print("  ⚡ Optimized for various content sizes")
print("  🔧 Detailed debugging and progress feedback")


In [ ]:
# Cell 5: JSON Export Method
def save_to_json(self, responses: List[Dict], prompt: str, filename: str = None):
    """Save responses to a JSON file"""
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"openrouter_responses_{timestamp}.json"
    
    data = {
        "timestamp": datetime.now().isoformat(),
        "prompt": prompt,
        "responses": responses,
        "statistics": {
            "total_models": len(responses),
            "successful_responses": sum(1 for r in responses if r["success"]),
            "failed_responses": sum(1 for r in responses if not r["success"]),
            "total_tokens": sum(int(r["tokens_used"]) for r in responses if str(r["tokens_used"]).isdigit())
        }
    }
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    
    print(f"💾 JSON saved: {filename}")
    return filename

# Add to class
AIModelComparison.save_to_json = save_to_json

print("✅ JSON export method added!")


In [ ]:
# Cell 6: Setup API Key
# Get your free API key from: https://openrouter.ai/
api_key = os.getenv('OPENROUTER_API_KEY')

if not api_key:
    api_key = input("Enter your OpenRouter API key: ").strip()

if api_key:
    print("🔑 API key configured!")
    ai = AIModelComparison(api_key)
    print(f"🚀 AI assistant ready with {len(ai.models)} models!")
else:
    print("❌ Need API key to continue. Get one from: https://openrouter.ai/")


In [ ]:
# Cell 7: Interactive Mode
def interactive_comparison_enhanced():
    """Interactive mode with enhanced 2-column PDF"""
    if 'ai' not in globals():
        print("❌ AI assistant not found")
        return
    
    print("🎮 Enhanced Interactive AI Model Comparison")
    print("📋 New Feature: 2-Column PDF (Model | Full Response)")
    print("=" * 60)
    
    user_prompt = input("Enter your question: ").strip()
    
    if not user_prompt:
        print("❌ Empty prompt!")
        return
    
    try:
        max_tokens = int(input("Max tokens (default 800): ") or 800)
    except ValueError:
        max_tokens = 800
    
    # Query all models
    print("\n🤖 Querying all models...")
    results = ai.query_multiple_models(user_prompt, max_tokens)
    
    # Display results
    ai.display_responses(results, user_prompt)
    
    # Ask about saving
    save_files = input("\n💾 Generate enhanced 2-column PDF report? (y/n): ").lower().strip()
    
    if save_files == 'y':
        pdf_file = ai.save_to_pdf(results, user_prompt)
        json_file = ai.save_to_json(results, user_prompt)
        print(f"\n✅ Reports saved:")
        print(f"  📄 PDF (2-column): {pdf_file}")
        print(f"  💾 JSON: {json_file}")
    
    print("✨ Enhanced comparison completed!")

print("🎉 Enhanced interactive mode ready!")
print("💡 Call: interactive_comparison_enhanced()")



In [ ]:
# Cell 8: Execute interactive Mode
interactive_comparison_enhanced()